### Spark Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens dataset](https://grouplens.org/datasets/movielens/latest/)

In [0]:
dbutils.library.restartPython()

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
%matplotlib inline

%matplotlib inline is not supported in Databricks.
You can display matplotlib figures using display(). For an example, see https://docs.databricks.com/user-guide/visualizations/matplotlib-and-ggplot.html

In [0]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [0]:
#use the big dataset for SQL analysis and recommendation
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [0]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

In [0]:
movie_ratings=ratings_df.drop('timestamp')
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [0]:
#convert to pandas dataframe
movies_pdf = movies_df.toPandas()
ratings_pdf = ratings_df.toPandas()
links_pdf = links_df.toPandas()
tags_pdf = tags_df.toPandas()

In [0]:
#use the small data set for ALS training and grid search
s_movies_df = spark.read.load("/FileStore/tables/movie_small/movies.csv", format='csv', header = True)
s_ratings_df = spark.read.load("/FileStore/tables/movie_small/ratings.csv", format='csv', header = True)
s_links_df = spark.read.load("/FileStore/tables/movie_small/links.csv", format='csv', header = True)
s_tags_df = spark.read.load("/FileStore/tables/movie_small/tags.csv", format='csv', header = True)

In [0]:
s_movies_df.registerTempTable("s_movies")
s_ratings_df.registerTempTable("s_ratings")
s_links_df.registerTempTable("s_links")
s_tags_df.registerTempTable("s_tags")

In [0]:
s_movie_ratings=s_ratings_df.drop('timestamp')
s_movie_ratings = s_movie_ratings.withColumn("userId", s_movie_ratings["userId"].cast(IntegerType()))
s_movie_ratings = s_movie_ratings.withColumn("movieId", s_movie_ratings["movieId"].cast(IntegerType()))
s_movie_ratings = s_movie_ratings.withColumn("rating", s_movie_ratings["rating"].cast(FloatType()))

In [0]:
#convert to pandas dataframe
s_movies_pdf = s_movies_df.toPandas()
s_ratings_pdf = s_ratings_df.toPandas()
s_links_pdf = s_links_df.toPandas()
s_tags_pdf = s_tags_df.toPandas()

In [0]:
movies_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [0]:
ratings_df.show(5)

+------+-------+------+----------+
userId|movieId|rating| timestamp|
+------+-------+------+----------+
 1| 307| 3.5|1256677221|
 1| 481| 3.5|1256677456|
 1| 1091| 1.5|1256677471|
 1| 1257| 4.5|1256677460|
 1| 1449| 4.5|1256677264|
+------+-------+------+----------+
only showing top 5 rows

In [0]:
links_df.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [0]:
tags_df.show(5)

+------+-------+------------+----------+
userId|movieId| tag| timestamp|
+------+-------+------------+----------+
 14| 110| epic|1443148538|
 14| 110| Medieval|1443148532|
 14| 260| sci-fi|1442169410|
 14| 260|space action|1442169421|
 14| 318|imdb top 250|1442615195|
+------+-------+------------+----------+
only showing top 5 rows

In [0]:
#use spark dataframe to get unique movie id from movies dataframe
movies_df.select('movieId').distinct().count()

Out[16]: 58098

In [0]:
#use pandas dataframe to get unique movie id from movies dataframe
len(movies_pdf.movieId.unique())

Out[17]: 58098

In [0]:
#use spark dataframe to get num of rows in rating
ratings_df.count()

Out[18]: 27753444

In [0]:
#use pandas dataframe to get unique movie id from ratings dataframe
len(ratings_pdf.movieId.unique())

Out[19]: 53889

In [0]:
#use pandas dataframe
tmp1 = ratings_pdf.groupby('userId').count()['rating'].min()
tmp2 = ratings_pdf.groupby('movieId').count()['rating'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))


For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 1
Minimum number of ratings per movie is 1

In [0]:
display(ratings_df.groupBy("movieId").count())

movieId,count
296,92406
1090,18825
2294,12974
3210,9819
48738,6166
691,1272
115713,11667
5325,1134
4821,1423
2136,3434


In [0]:
#using spark dataframe
tmp1 = ratings_df.groupBy("movieId").count().filter('count == 1').count()
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

10155 out of 53889 movies are rated by only one user

sparkDF.count() and pandasDF.count() are not the exactly the same.
The first one returns the number of rows, and the second one returns the number of non NA/null observations for each column.

In [0]:
rating_c = ratings_pdf.groupby("movieId").count()

In [0]:
#using pandas dataframe
ratings_c = ratings_pdf.groupby("movieId").count()
tmp1 = ratings_c[ratings_c['rating'] == 1].count()['rating']
tmp2 = len(ratings_pdf['movieId'].unique())
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

10155 out of 53889 movies are rated by only one user

In [0]:
%sql
select * from ratings order by rating desc limit 10

userId,movieId,rating,timestamp
250184,4973,5.0,1024955231
250184,4280,5.0,990219385
250184,6299,5.0,1100637147
250184,4979,5.0,1015116730
250184,4272,5.0,1028590233
250184,5325,5.0,1024954902
250184,3996,5.0,984773224
250184,6016,5.0,1047512220
250184,4027,5.0,984773224
250184,4298,5.0,1047512065


## Part 1: Spark SQL and OLAP

### Q1: The number of Users

In [0]:
%sql 
select count(distinct userId) as num_of_users from ratings

num_of_users
283228


In [0]:
#use Sparkdata Frame
temp = ratings_df.select('userId').distinct().count()
print(temp)

283228

### Q2: The number of Movies

In [0]:
%sql 
select count(distinct movieId) as num_of_movies from ratings

num_of_movies
53889


### Q3:  How many movies are rated by users? List movies not rated before

In [0]:
%sql 
select count(distinct movieId) as num_of_movies from movies

num_of_movies
58098


In [0]:
%sql 
select movieId, title, genres from movies
where movies.movieId not in
(select distinct movieId from ratings)

movieId,title,genres
25817,Break of Hearts (1935),Drama|Romance
26361,Baby Blue Marine (1976),Drama
27153,Can't Be Heaven (Forever Together) (2000),Children|Comedy|Drama|Romance
27433,Bark! (2002),Comedy|Drama
31945,Always a Bridesmaid (2000),Documentary
52696,"Thousand and One Nights, A (1001 Nights) (1945)",Adventure
58209,Alex in Wonder (Sex and a Girl) (2001),Comedy|Drama
60234,"Shock, The (1923)",Crime|Drama
69565,Bling: A Planet Rock (2007),Documentary
69834,Agency (1980),Drama


In [0]:
%sql 
select count(distinct movieId) from movies
where movies.movieId not in
(select distinct movieId from ratings)

count(DISTINCT movieId)
4209


### Q4: List Movie Genres

In [0]:
%sql
select distinct genres from movies

genres
Comedy|Horror|Thriller
Adventure|Sci-Fi|Thriller
Action|Adventure|Drama|Fantasy
Action|Drama|Horror
Comedy|Drama|Horror|Thriller
Action|Animation|Comedy|Sci-Fi
Fantasy|Musical|Mystery
Adventure|Mystery|Sci-Fi
Children|Comedy|Drama|Fantasy|Sci-Fi
Action|Adventure|Comedy|Horror|Thriller


In [0]:
%sql
select distinct explode(split(genres,'[|]')) genres  from movies order by genres


genres
(no genres listed)
Action
Adventure
Animation
Children
Comedy
Crime
Documentary
Drama
Fantasy


### Q5: Movie for Each Category

In [0]:
%sql
select Category, count(*) as num_of_movies
from
(select title, explode(split(genres,'[|]')) Category  from movies) a
group by Category
order by Category

Category,num_of_movies
(no genres listed),4266
Action,7130
Adventure,4067
Animation,2663
Children,2749
Comedy,15956
Crime,5105
Documentary,5118
Drama,24144
Fantasy,2637


In [0]:
%sql 
select Category, collect_list(title) as list_of_movies
from
(select explode(split(genres,'[|]')) Category, title from movies)a
group by Category

Category list_of_movies Crime List(Heat (1995), Casino (1995), Money Train (1995), Get Shorty (1995), Copycat (1995), Assassins (1995), Shanghai Triad (Yao a yao yao dao waipo qiao) (1995), Dead Man Walking (1995), Dead Presidents (1995), Usual Suspects, The (1995), Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996), Crossing Guard, The (1995), Things to Do in Denver When You're Dead (1995), Hate (Haine, La) (1995), Bottle Rocket (1996), Taxi Driver (1976), Rumble in the Bronx (Hont faan kui) (1995), Young Poisoner's Handbook, The (1995), Bad Boys (1995), Amateur (1994), Batman Forever (1995), Clockers (1995), Devil in a Blue Dress (1995), Die Hard: With a Vengeance (1995), Doom Generation, The (1995), Hackers (1995), Judge Dredd (1995), Net, The (1995), Strange Days (1995), Bushwhacked (1995), The Glass Shield (1994), Heavenly Creatures (1994), Kiss of Death (1995), Little Odessa (1994), New Jersey Drive (1995), New York Cop (Nyû Yôku no koppu) (1993), Natural Born Killers (1994), Once Were Warriors (1994), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Pulp Fiction (1994), Shawshank Redemption, The (1994), Tales from the Hood (1995), Clear and Present Danger (1994), Crow, The (1994), Mask, The (1994), Bad Company (1995), Faster Pussycat! Kill! Kill! (1965), Jason's Lyric (1994), Two Crimes (Dos crímenes) (1995), Above the Rim (1994), Beverly Hills Cop III (1994), Carlito's Way (1993), Fresh (1994), Getaway, The (1994), Guilty as Sin (1993), Hard Target (1993), Jimmy Hollywood (1994), Judgment Night (1993), Killing Zoe (1994), Menace II Society (1993), Perfect World, A (1993), RoboCop 3 (1993), Romeo Is Bleeding (1993), Serial Mom (1994), Striking Distance (1993), Trial by Jury (1994), True Romance (1993), Escort, The (Scorta, La) (1993), Batman (1989), Silence of the Lambs, The (1991), Love and a .45 (1994), Fargo (1996), Primal Fear (1996), Courage Under Fire (1996), Purple Noon (Plein soleil) (1960), Coup de torchon (Clean Slate) (1981), Substitute, The (1996), Mulholland Falls (1996), Captives (1994), Switchblade Sisters (1975), Heaven's Prisoners (1996), Original Gangstas (1996), Striptease (1996), Trainspotting (1996), American Buffalo (1996), Ransom (1996), Cyclo (Xich lo) (1995), Godfather, The (1972), Supercop (Police Story 3: Supercop) (Jing cha gu shi III: Chao ji jing cha) (1992), Bound (1996), Carpool (1996), Kansas City (1996), Killer: A Journal of Murder (1995), Nothing to Lose (1994), Supercop 2 (Project S) (Chao ji ji hua) (1993), Bulletproof (1996), Some Like It Hot (1959), Charade (1963), To Catch a Thief (1955), Laura (1944), Thin Man, The (1934), Angel on My Shoulder (1946), They Made Me a Criminal (1939), Beat the Devil (1953), Last Man Standing (1996), Set It Off (1996), 2 Days in the Valley (1996), Curdled (1996), Freeway (1996), Die Hard (1988), Normal Life (1996), Crossfire (1947), Murder, My Sweet (1944), Fish Called Wanda, A (1988), Bonnie and Clyde (1967), Dial M for Murder (1954), Reservoir Dogs (1992), Basic Instinct (1992), Funeral, The (1996), North Star (a.k.a. Tashunga) (1995), Return of the Pink Panther, The (1975), Wallace & Gromit: The Wrong Trousers (1993), He Walked by Night (1948), Grifters, The (1990), Tie Me Up! Tie Me Down! (¡Átame!) (1990), Clockwork Orange, A (1971), Goodfellas (1990), Killer, The (Die xue shuang xiong) (1989), Psycho (1960), Godfather: Part II, The (1974), Once Upon a Time in America (1984), Sting, The (1973), Miller's Crossing (1990), Touch of Evil (1958), Femme Nikita, La (Nikita) (1990), Chinatown (1974), M (1931), Manchurian Candidate, The (1962), Fried Green Tomatoes (1991), Big Sleep, The (1946), Mad Dog Time (1996), Cape Fear (1962), Blood and Wine (Blood & Wine) (1996), Albino Alligator (1996), Batman Returns (1992), Sneakers (1992), Night Falls on Manhattan (1996), Beavis and Butt-Head Do America (1996), La Cérémonie (1995), Hamlet (1996), Thieves (Voleurs, Les) (1996), Murder at 1600 (1997), Metro (1997), Gr

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, so let's reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [0]:
#Create test and train set from small dataset
(training,test)=s_movie_ratings.randomSplit([0.8,0.2])

In [0]:
#Create ALS model
als = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId",ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

In [0]:
# Define evaluator as RMSE
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.06917603975573

In [0]:
#Tune model using ParamGridBuilder
paramGrid = ParamGridBuilder() \
                    .addGrid(als.rank, [4, 8, 12]) \
                    .addGrid(als.maxIter, [10]) \
                    .addGrid(als.regParam, [0.05, 0.1, 0.2, 0.5]) \
                    .build()

In [0]:
# Build Cross validation 
cv = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator)

In [0]:
#Fit ALS model to training data
cvModel = cv.fit(training)

In [0]:
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = cvModel.bestModel

In [0]:
print ("**Best Model**")
print (" Rank:", best_model.rank)
print (" MaxIter:", best_model._java_obj.parent().getMaxIter())
print (" RegParam:", best_model._java_obj.parent().getRegParam())

### Model testing
And finally, make a prediction and check the testing error.

In [0]:
#use the best model parameters from grid search and cross validation
als = ALS(maxIter=10, regParam=0.2, rank=8,
          userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
best_model = als.fit(training)

In [0]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [0]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))

RMSE = 0.8775996422526896

In [0]:
predictions.show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 602| 471| 4.0| 3.2064168|
 603| 471| 4.0| 3.1276453|
 474| 471| 3.0| 3.2143643|
 500| 471| 1.0| 2.6335795|
 610| 471| 4.0| 3.2967114|
 520| 471| 5.0| 3.4988132|
 273| 471| 5.0| 3.7265475|
 448| 471| 4.0| 3.0764804|
 216| 471| 3.0| 3.156987|
 411| 471| 4.0| 2.9362526|
 414| 471| 5.0| 3.2362506|
 609| 833| 3.0| 1.5750232|
 159| 1088| 4.0| 3.032431|
 599| 1088| 2.5| 2.4353962|
 474| 1088| 3.5| 2.9101121|
 84| 1088| 3.0| 3.204459|
 525| 1088| 4.5| 3.0812328|
 600| 1088| 3.5| 2.4489245|
 325| 1238| 4.0| 4.045293|
 216| 1238| 5.0| 3.8103461|
+------+-------+------+----------+
only showing top 20 rows

### Model apply and see the performance

In [0]:
#use the trained model for big dataset
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 1.1543848199258788

In [0]:
alldata.registerTempTable("alldata")

In [0]:
%sql
select title, movieId from movies where movieId in
(select movieId from ratings where userId = 232 and rating >= 4)

title,movieId
Babe (1995),34
Hamlet (1990),3723
Carrington (1995),35
Toy Story That Time Forgot (2014),120474
Now and Then (1995),27
Othello (1995),26
It Takes Two (1995),38
Toy Story (1995),1
Balto (1995),13
Toy Story 3 (2010),78499


## Generate movie recommendation for user. 
you can choose some users to recommend the moives

In [0]:
#refit the model use all big data, for later recommendation use, important.
'''
https://spark.apache.org/docs/latest/ml-collaborative-filtering.html
If the rating matrix is derived from another source of information (i.e. it is inferred from other signals), you can set implicitPrefs to True to get better results
'''
'''als = ALS(maxIter=10, regParam=0.2, rank=8, implicitPrefs=True, 
          userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
          '''
als = ALS(maxIter=10, regParam=0.2, rank=8, implicitPrefs=True, 
          userCol="userId", itemCol="movieId", ratingCol="rating")
best_model = als.fit(movie_ratings)

In [0]:
#recommend 10 movies for all users
recomd_u = best_model.recommendForAllUsers(10)

In [0]:
display(recomd_u)

userId,recommendations
148,"List(List(62, 0.7868141), List(736, 0.77717054), List(141, 0.7756154), List(36, 0.7687032), List(25, 0.7637651), List(1073, 0.7559014), List(3, 0.7542067), List(648, 0.7452777), List(7, 0.738238), List(780, 0.736946))"
463,"List(List(296, 0.69881463), List(150, 0.690071), List(590, 0.6892074), List(593, 0.67417145), List(457, 0.67142355), List(318, 0.6631056), List(356, 0.65788364), List(592, 0.6503092), List(380, 0.64555633), List(480, 0.6204863))"
471,"List(List(2959, 1.2466464), List(2571, 1.2306089), List(7153, 1.2257848), List(4993, 1.2199664), List(5952, 1.2021577), List(58559, 1.1994581), List(79132, 1.1776949), List(318, 1.1706325), List(7361, 1.1184305), List(4226, 1.1150364))"
496,"List(List(260, 0.27091765), List(608, 0.268359), List(780, 0.26329416), List(32, 0.2610045), List(1, 0.2588848), List(1210, 0.24645767), List(648, 0.24478239), List(733, 0.23847869), List(858, 0.23522969), List(736, 0.22759134))"
833,"List(List(912, 0.11986438), List(969, 0.11609439), List(1304, 0.11465929), List(919, 0.11429093), List(908, 0.11406867), List(913, 0.11232535), List(1234, 0.11153993), List(1197, 0.11117274), List(1230, 0.111056946), List(1247, 0.11071422))"
1088,"List(List(1197, 0.06522084), List(912, 0.06085393), List(2396, 0.059230275), List(919, 0.05908241), List(2858, 0.058431458), List(1196, 0.057389546), List(1198, 0.057155795), List(1307, 0.056976814), List(260, 0.056861363), List(858, 0.05584131))"
1238,"List(List(590, 0.11646928), List(150, 0.11625482), List(592, 0.11481505), List(356, 0.112738796), List(588, 0.1113656), List(380, 0.11084168), List(457, 0.11060902), List(318, 0.10982719), List(480, 0.10783344), List(595, 0.10656832))"
1342,"List(List(2167, 0.014917935), List(1215, 0.01459178), List(1676, 0.014525402), List(1356, 0.01433814), List(1320, 0.014187472), List(1690, 0.014135936), List(1552, 0.014078544), List(3527, 0.014031528), List(1370, 0.013931099), List(1876, 0.013726668))"
1580,"List(List(260, 0.3697894), List(2571, 0.34726083), List(1210, 0.3412642), List(1196, 0.33852488), List(858, 0.336176), List(1198, 0.32728976), List(2959, 0.31447008), List(2858, 0.30228248), List(1270, 0.3014924), List(4993, 0.29916024))"
1591,"List(List(590, 0.97797626), List(150, 0.9721216), List(592, 0.96421504), List(380, 0.9613294), List(457, 0.9338364), List(349, 0.9300454), List(165, 0.92582583), List(153, 0.9168564), List(588, 0.91670287), List(344, 0.9039729))"


In [0]:
user_subset1 = movie_ratings.where(movie_ratings.userId == 12).select('userId').distinct()
recomd1 = best_model.recommendForUserSubset(user_subset1, 10).toPandas()

/databricks/spark/python/pyspark/sql/dataframe.py:2160: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: ArrayType(StructType(List(StructField(movieId,IntegerType,true),StructField(rating,FloatType,true))),true)
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
display(recomd1)

userId,recommendations
12,"List(List(1, 0.12869246304035187), List(1073, 0.1283300518989563), List(260, 0.12086264044046402), List(1393, 0.12081946432590485), List(2396, 0.11984322965145111), List(1210, 0.11936143785715103), List(1197, 0.11730700731277466), List(780, 0.11683876812458038), List(1307, 0.11385807394981384), List(1265, 0.11251908540725708))"


In [0]:
print('Recommended 10 movies for user with Id = 12:')
for recomd in recomd1['recommendations']:
  for movie in recomd:
    #print(movies_pdf[movies_pdf.movieId.astype(int) == movie[0]].loc[:,['movieId', 'title']], row.names = FALSE)
    #print('Recommended movie ID:', movie[0])
    print(movies_pdf[movies_pdf.movieId.astype(int) == movie[0]]['title'].to_string(index=False, header=False), movies_pdf[movies_pdf.movieId.astype(int) == movie[0]]['genres'].to_string(index=False, header=False))

Recommended 10 movies for user with Id = 12:
 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
 Willy Wonka & the Chocolate Factory (1971) Children|Comedy|Fantasy|Musical
 Star Wars: Episode IV - A New Hope (1977) Action|Adventure|Sci-Fi
 Jerry Maguire (1996) Drama|Romance
 Shakespeare in Love (1998) Comedy|Drama|Romance
 Star Wars: Episode VI - Return of the Jedi (1983) Action|Adventure|Sci-Fi
 Princess Bride, The (1987) Action|Adventure|Comedy|Fantasy|Romance
 Independence Day (a.k.a. ID4) (1996) Action|Adventure|Sci-Fi|Thriller
 When Harry Met Sally... (1989) Comedy|Romance
 Groundhog Day (1993) Comedy|Fantasy|Romance

In [0]:
%sql
select userId, title, rating, genres from movies join ratings on movies.movieId = ratings.movieId where ratings.userId = 12 order by rating desc limit 10

userId,title,rating,genres
12,Platoon (1986),3.5,Drama|War
12,Office Space (1999),3.5,Comedy|Crime
12,Lethal Weapon 2 (1989),3.0,Action|Comedy|Crime|Drama
12,"Bridges of Madison County, The (1995)",3.0,Drama|Romance
12,To Die For (1995),3.0,Comedy|Drama|Thriller
12,Romancing the Stone (1984),3.0,Action|Adventure|Comedy|Romance
12,Almost Famous (2000),3.0,Drama
12,Grease (1978),3.0,Comedy|Musical|Romance
12,Pocahontas (1995),2.5,Animation|Children|Drama|Musical|Romance
12,Nine Months (1995),2.5,Comedy|Romance


In [0]:
user_subset2 = movie_ratings.where(movie_ratings.userId == 232)
#best_model.recommendForUserSubset(user_subset2, 10).toPandas()['recommendations']
recomd2 = best_model.recommendForUserSubset(user_subset2, 10).toPandas()

/databricks/spark/python/pyspark/sql/dataframe.py:2160: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: ArrayType(StructType(List(StructField(movieId,IntegerType,true),StructField(rating,FloatType,true))),true)
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
display(recomd2)

userId,recommendations
232,"List(List(356, 0.1931414157152176), List(588, 0.18697179853916168), List(595, 0.18611793220043182), List(1, 0.1818205565214157), List(318, 0.1815967708826065), List(364, 0.18056941032409668), List(17, 0.17492637038230896), List(150, 0.1733342707157135), List(597, 0.1717706024646759), List(539, 0.1716485470533371))"


In [0]:
print('Recommended 10 movies for user with Id = 232:')
for recomd in recomd2['recommendations']:
  for movie in recomd:
    print(movies_pdf[movies_pdf.movieId.astype(int) == movie[0]]['title'].to_string(index=False, header=False), movies_pdf[movies_pdf.movieId.astype(int) == movie[0]]['genres'].to_string(index=False, header=False))

Recommended 10 movies for user with Id = 232:
 Forrest Gump (1994) Comedy|Drama|Romance|War
 Aladdin (1992) Adventure|Animation|Children|Comedy|Musical
 Beauty and the Beast (1991) Animation|Children|Fantasy|Musical|Romance|IMAX
 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
 Shawshank Redemption, The (1994) Crime|Drama
 Lion King, The (1994) Adventure|Animation|Children|Drama|Musical|IMAX
 Sense and Sensibility (1995) Drama|Romance
 Apollo 13 (1995) Adventure|Drama|IMAX
 Pretty Woman (1990) Comedy|Romance
 Sleepless in Seattle (1993) Comedy|Drama|Romance

In [0]:
%sql
select userId, title, rating, genres from movies join ratings on movies.movieId = ratings.movieId where ratings.userId = 232 order by rating desc limit 10

userId,title,rating,genres
232,Babe (1995),5.0,Children|Drama
232,Toy Story That Time Forgot (2014),5.0,Animation|Children
232,Toy Story (1995),5.0,Adventure|Animation|Children|Comedy|Fantasy
232,Now and Then (1995),5.0,Children|Drama
232,Othello (1995),4.5,Drama
232,Balto (1995),4.0,Adventure|Animation|Children
232,It Takes Two (1995),4.0,Children|Comedy
232,Carrington (1995),4.0,Drama|Romance
232,Hamlet (1990),4.0,Drama
232,Toy Story 3 (2010),4.0,Adventure|Animation|Children|Comedy|Fantasy|IMAX


In [0]:
user_subset3 = movie_ratings.where(movie_ratings.userId == 575)
recomd3 = best_model.recommendForUserSubset(user_subset3, 10).toPandas()

/databricks/spark/python/pyspark/sql/dataframe.py:2160: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: ArrayType(StructType(List(StructField(movieId,IntegerType,true),StructField(rating,FloatType,true))),true)
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

In [0]:
display(recomd3)

userId,recommendations
575,"List(List(6016, 0.13914644718170166), List(3949, 0.1370687633752823), List(6711, 0.13525713980197906), List(4848, 0.1323132961988449), List(5902, 0.13068969547748566), List(7361, 0.12972886860370636), List(4235, 0.1266142874956131), List(55820, 0.1251455694437027), List(44555, 0.12411986291408539), List(4973, 0.12391820549964905))"


In [0]:
print('Recommended 10 movies for user with Id = 575:')
for recomd in recomd3['recommendations']:
  for movie in recomd:
    print(movies_pdf[movies_pdf.movieId.astype(int) == movie[0]]['title'].to_string(index=False, header=False),  movies_pdf[movies_pdf.movieId.astype(int) == movie[0]]['genres'].to_string(index=False, header=False))
    

Recommended 10 movies for user with Id = 575:
 City of God (Cidade de Deus) (2002) Action|Adventure|Crime|Drama|Thriller
 Requiem for a Dream (2000) Drama
 Lost in Translation (2003) Comedy|Drama|Romance
 Mulholland Drive (2001) Crime|Drama|Film-Noir|Mystery|Thriller
 Adaptation (2002) Comedy|Drama|Romance
 Eternal Sunshine of the Spotless Mind (2004) Drama|Romance|Sci-Fi
 Amores Perros (Love's a Bitch) (2000) Drama|Thriller
 No Country for Old Men (2007) Crime|Drama
 Lives of Others, The (Das leben der Anderen) (... Drama|Romance|Thriller
 Amelie (Fabuleux destin d'Amélie Poulain, Le) ... Comedy|Romance

In [0]:
%sql
select userId, title, rating, genres from movies join ratings on movies.movieId = ratings.movieId where ratings.userId = 575 order by rating desc limit 10

userId,title,rating,genres
575,City of God (Cidade de Deus) (2002),5.0,Action|Adventure|Crime|Drama|Thriller
575,Sin Nombre (2009),5.0,Crime|Drama|Thriller
575,Monster (2003),4.5,Crime|Drama
575,"Maria Full of Grace (Maria, Llena eres de gracia) (2004)",4.5,Crime|Drama
575,Let the Right One In (Låt den rätte komma in) (2008),4.5,Drama|Fantasy|Horror|Romance
575,Sexy Beast (2000),4.0,Crime|Drama
575,"Downfall (Untergang, Der) (2004)",4.0,Drama|War
575,Sophie's Choice (1982),4.0,Drama
575,Insomnia (2002),3.5,Action|Crime|Drama|Mystery|Thriller
575,Yellow Submarine (1968),3.5,Adventure|Animation|Comedy|Fantasy|Musical


## Generate user recommendations for movie

In [0]:
#recommend 10 users for all movies
recomd_m =best_model.recommendForAllItems(10)

In [0]:
display(recomd_m)

movieId,recommendations
148,"List(List(63783, 0.1622146), List(242683, 0.15752669), List(234257, 0.15556538), List(61614, 0.15521207), List(117490, 0.15258026), List(158002, 0.15225318), List(275204, 0.14880154), List(119138, 0.14518434), List(133398, 0.14506495), List(4796, 0.14431834))"
463,"List(List(242683, 0.23497523), List(158002, 0.23201655), List(63783, 0.2319092), List(117490, 0.22595134), List(133398, 0.22226153), List(60950, 0.21571434), List(61614, 0.21413128), List(253511, 0.21320954), List(191028, 0.21258646), List(263685, 0.21244621))"
471,"List(List(51571, 1.1258805), List(64180, 1.1258088), List(222512, 1.1067255), List(61614, 1.1051285), List(4027, 1.0960349), List(234257, 1.0854701), List(242683, 1.0826643), List(55508, 1.079572), List(133398, 1.0758718), List(158002, 1.0747339))"
496,"List(List(234257, 0.2125623), List(63783, 0.21114734), List(242683, 0.2067823), List(61614, 0.20577873), List(275204, 0.20346825), List(117490, 0.19976257), List(158002, 0.19892025), List(119138, 0.19869293), List(4796, 0.19792932), List(64180, 0.19396009))"
833,"List(List(158194, 0.32262716), List(275244, 0.32154945), List(137652, 0.31445602), List(83579, 0.31331256), List(211520, 0.31153002), List(37046, 0.31094062), List(29973, 0.31011322), List(108820, 0.30989295), List(109040, 0.30897206), List(184203, 0.30800995))"
1088,"List(List(116110, 1.2499892), List(255993, 1.2058072), List(67488, 1.177779), List(133398, 1.1641905), List(236327, 1.1484185), List(92773, 1.1461512), List(12951, 1.142095), List(239948, 1.1409265), List(102235, 1.1394608), List(208719, 1.1383941))"
1238,"List(List(51571, 0.9565377), List(64180, 0.94884425), List(89499, 0.9294701), List(234257, 0.9225755), List(278718, 0.9209942), List(275204, 0.91845423), List(251512, 0.91314507), List(63783, 0.9043328), List(55508, 0.9013114), List(49815, 0.89572924))"
1342,"List(List(158002, 0.8487501), List(275244, 0.8395741), List(253511, 0.8282084), List(242683, 0.82248086), List(60950, 0.8074956), List(48083, 0.8053041), List(10379, 0.79758584), List(7705, 0.79599464), List(137652, 0.7912189), List(227459, 0.78102136))"
1580,"List(List(107560, 1.2478739), List(273250, 1.24579), List(48196, 1.2455403), List(66948, 1.2380371), List(56484, 1.2324184), List(81659, 1.2293804), List(48866, 1.2293248), List(26676, 1.2289238), List(181337, 1.2177596), List(262446, 1.2175303))"
1591,"List(List(10379, 0.93992215), List(275244, 0.93372047), List(19309, 0.85696155), List(122978, 0.85058624), List(117096, 0.84793013), List(175615, 0.8465464), List(71277, 0.846133), List(12152, 0.8452715), List(205855, 0.84472096), List(239643, 0.8418667))"


In [0]:
item_subset1 = movie_ratings.where(movie_ratings.movieId == 10).select('movieId').distinct()
display(item_subset1)

movieId
10


In [0]:
print('recommend movie with ID = 10:', movies_pdf[movies_pdf.movieId.astype(int) == 10]['title'].to_string(index=False, header=False))

recommend movie with ID = 10: GoldenEye (1995)

In [0]:
recomend1 = best_model.recommendForItemSubset(item_subset1, 10).toPandas()
display(recomend1)

movieId,recommendations
10,"List(List(240605, 1.0687283277511597), List(170891, 1.0609444379806519), List(91005, 1.0593087673187256), List(4060, 1.0575007200241089), List(74014, 1.0543633699417114), List(276542, 1.0537500381469727), List(197401, 1.0520024299621582), List(135513, 1.0516594648361206), List(275616, 1.0515069961547852), List(42704, 1.0513238906860352))"
